In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score, classification_report, log_loss, make_scorer
import seaborn as sn
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
import sklearn

In [128]:
df = pd.read_csv('grad.csv')
df.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [129]:
y = df.pop('admit')
X = df

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
gre     400 non-null int64
gpa     400 non-null float64
rank    400 non-null int64
dtypes: float64(1), int64(2)
memory usage: 9.5 KB


In [131]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=21)
# If not scaling ...
# Convergence Warning: The max_iter was reached which means the coef_ did not converge
scalar = StandardScaler().fit(X_train)
X_train = scalar.transform(X_train)
X_test = scalar.transform(X_test)

C:\Users\en7qltl\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\en7qltl\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """
C:\Users\en7qltl\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [132]:
#X_trainp, X_val, y_trainp, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=21)
# If not scaling ...
# Convergence Warning: The max_iter was reached which means the coef_ did not converge
#scalar = StandardScaler().fit(X_trainp)
#X_trainp = scalar.transform(X_trainp)
#X_val = scalar.transform(X_val)

# Logistic Regression

In [133]:
params = {
          'penalty': ['l2'], 
          'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
          'fit_intercept': [True, False],
          'C': np.logspace(-3,3)
         }
gs = GridSearchCV(LogisticRegression(), param_grid=params, scoring='neg_log_loss', cv=10, n_jobs=5).fit(X_train, y_train)
gs

C:\Users\en7qltl\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=5,
       param_grid={'penalty': ['l2'], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'fit_intercept': [True, False], 'C': array([1.00000e-03, 1.32571e-03, 1.75751e-03, 2.32995e-03, 3.08884e-03,
       4.09492e-03, 5.42868e-03, 7.19686e-03, 9.54095e-03, 1.26486e-02,
       1.67683e-02, 2.2230... 1.84207e+02, 2.44205e+02,
       3.23746e+02, 4.29193e+02, 5.68987e+02, 7.54312e+02, 1.00000e+03])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_log_loss', verbose=0)

In [134]:
gs.best_estimator_

LogisticRegression(C=0.0517947467923121, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [135]:
gs.best_params_

{'C': 0.0517947467923121,
 'fit_intercept': True,
 'penalty': 'l2',
 'solver': 'lbfgs'}

In [136]:
model = LogisticRegression(**gs.best_params_).fit(X_train, y_train)

In [137]:
y_hat = model.predict(X_test)
p_hat = model.predict_proba(X_test)[:,1]

In [138]:
log_loss(y_test, p_hat)

0.5401314955826813

In [139]:
baseline = 1 - (y_test.sum() / y_test.size)
accuracy = accuracy_score(y_test, y_hat)
baseline, accuracy

(0.7424242424242424, 0.75)

In [140]:
precision_score(y_test, y_hat)

0.5555555555555556

In [141]:
recall_score(y_test, y_hat)

0.14705882352941177

# Random Forest

In [142]:
params = {
            'n_estimators': [200],
            'criterion': ['gini', 'entropy'],
            'max_depth': range(1, 15),
            'max_features': ['sqrt', 'log2']
         }
gs = GridSearchCV(RandomForestClassifier(), param_grid=params, scoring='neg_log_loss', cv=5, n_jobs=-1).fit(X_train, y_train)
gs

C:\Users\en7qltl\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [200], 'criterion': ['gini', 'entropy'], 'max_depth': range(1, 15), 'max_features': ['sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_log_loss', verbose=0)

In [143]:
gs.best_params_

{'criterion': 'gini',
 'max_depth': 3,
 'max_features': 'sqrt',
 'n_estimators': 200}

In [144]:
rfc = RandomForestClassifier(**gs.best_params_).fit(X_train, y_train)
yhat = rfc.predict(X_test)
phat = rfc.predict_proba(X_test)[:,1]

In [145]:
accuracy_score(y_test, yhat), precision_score(y_test, yhat), recall_score(y_test, yhat), roc_auc_score(y_test, phat)

(0.7727272727272727, 0.7, 0.20588235294117646, 0.6664165666266507)

# Naive Bayes

In [146]:
GaussianNB().fit(X_train, y_train).score(X_test,y_test) # in tis case, this is the model to use as all features look like real numbers with normal dist

0.7954545454545454

In [147]:
BernoulliNB().fit(X_train,y_train).score(X_test,y_test) # you use this when features possibel values are 0 and 1 (only two outcomes)

0.7727272727272727

# Best model is GaussianNB with 0.79 score